In [57]:
from dataclasses import dataclass
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

import seaborn as sns
sns.set()

### Helper

In [72]:
def plot_decision_boundary(classifier, X, y, N = 10 , ax = None ):
    '''Utility function to plot decision boundary and scatter plot of data'''
    x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
    y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
    xx, yy = np.meshgrid( np.linspace(x_min, x_max, N), np.linspace(y_min, y_max, N))
    classes = len(np.unique(y))
    
    #Check what methods are available
    if hasattr(classifier, "predict"):
        zz = np.array( [classifier.predict(np.array([xi,yi]).reshape(1,-1)) for  xi, yi in zip(np.ravel(xx), np.ravel(yy)) ] )
    elif hasattr(classifier, "predict_probabilities"):
        zz = np.array( [classifier.predict_proba(np.array([xi,yi]).reshape(1,-1))[:,1] for  xi, yi in zip(np.ravel(xx), np.ravel(yy)) ] )
    else :
        zz = np.array( [classifier(np.array([xi,yi]).reshape(1,-1)) for  xi, yi in zip(np.ravel(xx), np.ravel(yy)) ] )
            
    # reshape result and plot
    Z = zz.reshape(xx.shape)
    
    
    #Get current axis and plot
    if ax is None:
        ax = plt.gca()
    ax.contourf(xx, yy, Z, classes-1, cmap='viridis', alpha=.2)
    ax.contour(xx, yy, Z,  classes-1, cmap='viridis')
    ax.scatter(X[:,0],X[:,1],c = classifier.predict(X), cmap = 'viridis')
    ax.set_xlabel('$X_1$')
    ax.set_ylabel('$X_2$')

### Global variables

In [73]:
gmm_samples_path = "./GMM.in"

In [74]:
def load_data() -> np.ndarray:
    data = []
    with open(gmm_samples_path) as f:
        for line in f:
            [x, y] = line.split(" ")
            x = float(x)
            y = float(y)
            data.append([x, y])
    
    return np.array(data)

In [75]:
d = load_data()

In [83]:
class GMM:
    def __init__(self, num_components: int = 4, iter_limit: int = 100, eps: float = 0.001):
        self.num_components = num_components
        self.iter_limit = iter_limit
        self.eps = eps
        
        # Inner parameters
        self.means = None
        self.covariances = None
        self.log_likelihoods = None
    
    def fit(self, X: np.ndarray) -> None:

        n, d = X.shape  ## n = datapoints, d = features
        k = self.num_components  ## K number of clusters

        # randomly initialize the starting means
        mu = X[np.random.choice(n, k, replace = False)]

        # initialize a covariance matrix for each gaussian
        Sigma = [np.eye(d)] * k

        # initialize the probability for each gaussian pi
        pi = np.array([1 / k] * k)

        # initialize responsibility matrix: n points for each gaussian
        W = np.zeros((n,k))

        # initialize list of log-likelihoods
        log_likelihoods = []

        # lambda function for gaussian pdf
        P = lambda m, s: multivariate_normal.pdf(X, mean = m, cov = s)
            
        while len(log_likelihoods) < self.iter_limit:

            # E step

            # nominator of responsibilities: j is the j-th gaussian
            for j in range(k):
                W[:, j] = pi[j] * P(mu[j], Sigma[j])

            # log likelihood computation (same as nominator of responsibilities)    
            l = np.sum(np.log(np.sum(W, axis = 1)))

            # store log likelihood in list
            log_likelihoods.append(l)

            # compute W matrix by dividing by denominator (the sum along j) 
            W = (W.T / W.sum(axis = 1)).T

            # sum of w^i entries along j (used for parameter updates)
            # these are the soft weighted number of datapoints belonging to each gaussian
            W_s = np.sum(W, axis = 0)


            # M step

            for j in range(k):

                ## Update means
                mu[j] = (1. / W_s[j]) * np.sum(W[:, j] * X.T, axis = 1).T 

                ## Update covariances
                Sigma[j] = ((W[:,j] * ((X - mu[j]).T)) @ (X - mu[j])) / W_s[j]

                ## Update probabilities of each gaussian
                pi[j] = W_s[j] / n

            # check for convergence
            if len(log_likelihoods) < 2: continue
            if np.abs(l - log_likelihoods[-2]) < self.eps: break

        self.means = mu
        self.covariances = Sigma
        self.log_likelihoods = log_likelihoods
        
    def predict_probabilities(self, x0: np.ndarray) -> np.ndarray:
        probs = np.array([ multivariate_normal.pdf(x0, mean = self.means[j], cov = self.covariances[j]) for j in range(self.num_components) ])
        return probs
    
    def predict(self, x0: np.ndarray) -> np.ndarray:
        probs = np.array([ multivariate_normal.pdf(x0, mean = self.means[j], cov = self.covariances[j]) for j in range(self.num_components) ])
        return np.argmax(probs, axis = 0)

In [84]:
gmm = GMM()
gmm.fit(d)


In [85]:
gmm.means

array([[-0.099163  , -0.06309399],
       [ 4.93536128, -0.02779437],
       [-2.85908732,  6.93598154],
       [-2.10300759, -5.12546242]])

In [86]:
gmm.covariances

[array([[0.87489786, 0.10117956],
        [0.10117956, 1.12586096]]),
 array([[1.82469838, 0.85857754],
        [0.85857754, 1.8899746 ]]),
 array([[ 2.68001822, -2.43363689],
        [-2.43363689,  4.83792193]]),
 array([[ 3.68868973, -1.10814283],
        [-1.10814283,  4.05763977]])]

In [87]:
# Label the points 
y = gmm.predict(x0 = d)

In [ ]:
plot_decision_boundary(gmm, d, y, N = 10 , ax = None )
plt.title('EM-Gausian Mixture Model')